# 1. Importing Libraries

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 2. Loading data

In [28]:
with open('Buddenbrooks1_Thomas-Mann_projekt-Gutenberg_34811-8.txt', 'r') as file:
    text = file.read()

# 3. Tokenizing the text to create sequence of words

In [29]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# 4. Creating input-output pairs

In [30]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# 5. Padding input sequences to have equal length

In [31]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,maxlen=max_sequence_len, padding='pre'))

# 6. Splitting sequences into input and output

In [32]:
X = input_sequences[:,:-1]
y = input_sequences[:,-1]

# 7. Converting the output to one-hot encode vectors

In [33]:
y = np.array(tf.keras.utils.to_categorical(y,num_classes=total_words))

# 8. Building Neural Network Architecture

In [34]:
model = Sequential()
model.add(Embedding(total_words,100,input_length = max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words,activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 15, 100)           2753300   
                                                                 
 lstm_1 (LSTM)               (None, 150)               150600    
                                                                 
 dense_1 (Dense)             (None, 27533)             4157483   
                                                                 
Total params: 7061383 (26.94 MB)
Trainable params: 7061383 (26.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


# 9. Compile and Train the model

In [35]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,epochs=100,verbose=1)

Epoch 1/100
6629/6629 [==============================] - 435s 65ms/step - loss: 7.3447 - accuracy: 0.0546
Epoch 2/100
6629/6629 [==============================] - 446s 67ms/step - loss: 6.5515 - accuracy: 0.0846
Epoch 3/100
6629/6629 [==============================] - 443s 67ms/step - loss: 6.0286 - accuracy: 0.1068
Epoch 4/100
6629/6629 [==============================] - 454s 68ms/step - loss: 5.5395 - accuracy: 0.1296
Epoch 5/100
6629/6629 [==============================] - 460s 69ms/step - loss: 5.0556 - accuracy: 0.1573
Epoch 6/100
6629/6629 [==============================] - 469s 71ms/step - loss: 4.5983 - accuracy: 0.1943
Epoch 7/100
6629/6629 [==============================] - 464s 70ms/step - loss: 4.1789 - accuracy: 0.2414
Epoch 8/100
6629/6629 [==============================] - 456s 69ms/step - loss: 3.8025 - accuracy: 0.2917
Epoch 9/100
6629/6629 [==============================] - 463s 70ms/step - loss: 3.4676 - accuracy: 0.3412
Epoch 10/100
6629/6629 [======================

6629/6629 [==============================] - 421s 63ms/step - loss: 0.8273 - accuracy: 0.8076
Epoch 78/100
6629/6629 [==============================] - 418s 63ms/step - loss: 0.8251 - accuracy: 0.8074
Epoch 79/100
6629/6629 [==============================] - 418s 63ms/step - loss: 0.8226 - accuracy: 0.8083
Epoch 80/100
6629/6629 [==============================] - 419s 63ms/step - loss: 0.8231 - accuracy: 0.8073
Epoch 81/100
6629/6629 [==============================] - 420s 63ms/step - loss: 0.8215 - accuracy: 0.8081
Epoch 82/100
6629/6629 [==============================] - 454s 68ms/step - loss: 0.8156 - accuracy: 0.8084
Epoch 83/100
6629/6629 [==============================] - 426s 64ms/step - loss: 0.8176 - accuracy: 0.8082
Epoch 84/100
6629/6629 [==============================] - 422s 64ms/step - loss: 0.8152 - accuracy: 0.8093
Epoch 85/100
6629/6629 [==============================] - 421s 63ms/step - loss: 0.8173 - accuracy: 0.8081
Epoch 86/100
6629/6629 [==========================

# 10. Saving the model

In [36]:
model.save('de_buddenbrooks-teil1_LSTM_model_v2_100epochs.h5')

# 11. Generating the next word predictions

In [53]:
from tensorflow.keras.models import load_model
# Load the saved model
model = load_model('de_buddenbrooks-teil1_LSTM_model_v2_100epochs.h5')
seed_text = "Die Gräfin"
next_words = 7

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 132ms/step
Die Gräfin konsulin antwortete tony und sogar mit widerwillen
